### Madipelli Rishitha 
### Extracting ER Data and converting the data to images and creating annotations

In [52]:
## parsed the XML files to get the ER-Readings and metadata. 
### Also generate the images of ER Readings 

import os
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from PIL import Image

# Parse the XML file and extract ER-Readings
def parse_er_readings_from_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    x_locations = []
    y_locations = []
    electrical_readings = []
    
    # Extract ER-Readings
    list_iter = ["ER-Readings", "NDE001Readings", "NDE001-Readings"]
    for i in list_iter:
        for er_reading in root.findall(".//" + i):
            y_location = float(er_reading.find('YLocation').text)
            x_location = float(er_reading.find('XLocation').text)
            electrical_reading = float(er_reading.find('ElectricalReading').text)
            
            y_locations.append(y_location)
            x_locations.append(x_location)
            electrical_readings.append(electrical_reading)
        
    return x_locations, y_locations, electrical_readings

# Create CSV files for ER-Readings and other tags
def create_csv_files(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    other_tags = []
    
    # Create CSV for ER-Readings
    er_csv_file_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(xml_file))[0]}_ER-Readings.csv")
    with open(er_csv_file_path, mode='w', newline='', encoding='utf-8') as er_csv_file:
        writer = csv.writer(er_csv_file)
        writer.writerow(['XLocation', 'YLocation', 'ElectricalReading'])
        
        x_locations, y_locations, electrical_readings = parse_er_readings_from_xml(xml_file)
        for x, y, er in zip(x_locations, y_locations, electrical_readings):
            writer.writerow([x, y, er])
    
    # Create CSV for other tags
    for child in root:
        if child.tag not in ["ER-Readings", "NDE001Readings", "NDE001-Readings"]:
            for element in child:
                other_tags.append([child.tag, element.tag, element.text])
                other_csv_file_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(xml_file))[0]}_metadata.csv")
                with open(other_csv_file_path, mode='w', newline='', encoding='utf-8') as other_csv_file:
                    writer = csv.writer(other_csv_file)
                    writer.writerow(['ParentTag', 'Tag', 'Value'])
                    writer.writerows(other_tags)

# Convert electrical readings to resistivity ranges
def categorize_resistivity(value):
    if value < 5:
        return 1  # Very high
    elif 5 <= value < 10:
        return 2  # High
    elif 10 <= value < 20:
        return 3  # Moderate
    elif 20 <= value < 40:
        return 4  # Low
    else:
        return 5  # Very Low

# Create an image from CSV
def create_image_from_csv(x, y, er, output_image):
    # Convert to DataFrame for easier manipulation
    data = pd.DataFrame({'XLocation': x, 'YLocation': y, 'ElectricalReading': er})
    data['ResistivityRange'] = data['ElectricalReading'].apply(categorize_resistivity)
    
    # Define colors for each resistivity range
    colors = ['darkred', 'red', 'orange', 'blue', 'silver']
    cmap = plt.cm.colors.ListedColormap(colors)
    
    # Create a contour plot with categorized resistivity levels
    fig, ax = plt.subplots(figsize=(20, 6))
    ax.set_aspect(aspect='auto')
    contour = plt.tricontourf(data['XLocation'], data['YLocation'], data['ResistivityRange'], levels=[1, 2, 3, 4, 5, 6], cmap=cmap)
    
    plt.xlabel('X Location')
    plt.ylabel('Y Location')
    plt.title('Contour Plot of Electrical Readings Converted to Resistivity Ranges')
    
    # Save the image in RGB format (3 channels)
    plt.savefig(output_image, format='png', dpi=300, bbox_inches='tight', pad_inches=0.1, transparent=False)
    plt.close()

    img = Image.open(output_image)
    rgb_img = img.convert('RGB')
    rgb_img.save(output_image)

# Directories
input_dir = '/Users/madipellirishitha/Downloads/data/Input/Electrical Resistivity'
output_dir_files = '/Users/madipellirishitha/Downloads/data/Output/ER/Files'
output_dir_images = '/Users/madipellirishitha/Downloads/data/Output/ER/Images'

# Create output directories if they don't exist
os.makedirs(output_dir_files, exist_ok=True)
os.makedirs(output_dir_images, exist_ok=True)

# Iterate over each directory and process XML files
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.xml'):
        xml_file_path = os.path.join(input_dir, filename)
        output_image_path = os.path.join(output_dir_images, f"{os.path.splitext(filename)[0]}.png")
        
        x_locations, y_locations, electrical_readings = parse_er_readings_from_xml(xml_file_path)
        create_image_from_csv(x_locations, y_locations, electrical_readings, output_image_path)
            
        # Create CSV files for ER-Readings and other tags
        create_csv_files(xml_file_path, output_dir_files)

print("Processing complete.")

Processing complete.


In [ ]:
### creating annotations.json file for training and validation

import cv2
import numpy as np
import os
import json

def save_masked_images_and_annotations(input_dir, output_dir, annotation_file):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    coco_format = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 1, "name": "corroded", "supercategory": "defect"}
        ]
    }

    annotation_id = 1
    image_id = 1  # Initialize an image ID counter
    for filename in os.listdir(input_dir):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # Adjust based on your image format

            # Construct the full path to the input image
            image_path = os.path.join(input_dir, filename)

            # Load image
            img = cv2.imread(image_path)
            height, width = img.shape[:2]

            # Convert image to HSV color space
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            # Define lower and upper bounds for the color of interest (e.g., dark red)
            lower_red = np.array([0, 50, 50])
            upper_red = np.array([10, 255, 150])

            # Threshold the HSV image to get only specified color
            mask = cv2.inRange(hsv, lower_red, upper_red)

            # Find contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Add image info to COCO format
            coco_format["images"].append({
                "id": image_id,
                "file_name": filename,
                "height": height,
                "width": width
            })

            # Process each contour
            for contour in contours:
                # Get bounding box coordinates
                x, y, w, h = cv2.boundingRect(contour)

                # Draw rectangle around the contour
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)

                # Label the region as 'corroded'
                label = 'corroded'
                cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Add annotation info to COCO format
                coco_format["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": 1,
                    "bbox": [x, y, w, h],
                    "area": w * h,
                    "iscrowd": 0
                })
                annotation_id += 1

            # Save the processed image with bounding boxes
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, img)

            print(f"Processed and saved: {output_path}")

            # Increment the image ID counter
            image_id += 1

    # Save annotations to JSON file
    with open(annotation_file, 'w') as f:
        json.dump(coco_format, f, indent=4)

# Example usage:
input_directory = '/Users/madipellirishitha/Downloads/data/Output/ER/split/train'
output_directory = '/Users/madipellirishitha/Downloads/data/Output/ER/v_op'
annotation_file = '/Users/madipellirishitha/Downloads/data/Output/ER/split/train/annotations.json'
save_masked_images_and_annotations(input_directory, output_directory, annotation_file)